# Compound Frequency Analysis 
This notebook performs the compound level analysis between the catalyst_inclusive and catalyst_lacking networks. We dive into the number of compounds being catalysed, the distribution of compound counts across each generation. We have also included descriptor functions to calculate descriptors for compounds whenever needed.

## Imports
Note: Please install rd-kit environment to import rdkit libarires. You may also have to install mordred.

In [2]:
import pandas as pd
import numpy as np

from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit import Chem

from matplotlib import pyplot as plt
import seaborn as sns
import pandas.plotting

from mordred import Calculator, descriptors
from IPython.display import display_html
from itertools import chain,cycle
import seaborn as sns


%matplotlib notebook

%pprint

ModuleNotFoundError: No module named 'rdkit'

These functions reduce the time.

In [ ]:
def load_data(reaction_name):
    '''This loads the reaaction csv data files and returns the old,
    new dataframes along with sets containing common and new compounds.'''
    old=pd.read_csv(f'{reaction_name}/old/{reaction_name}_output.csv', delimiter='\t') 
    new=pd.read_csv(f'{reaction_name}/new/{reaction_name}_output.csv', delimiter='\t')
    
    old_unique=set(old.loc[:,'ID'].values.tolist())
    new_unique=set(new.loc[:,'ID'].values.tolist())

    only_old=old_unique-new_unique #Set Difference Old - New
    only_new=new_unique-old_unique
    
    return [old, new,only_old, only_new]


def split_gens(dataframe):
    '''Split a single output dataframe into individual dataframes as per their generation.'''
    no_of_gens=dataframe.iloc[-1][0]
    gen_list=[]
    for g in range(no_of_gens+1):
        gen_list.append(dataframe.loc[dataframe['Gen']==g,:])
    return gen_list

def compare_for_shorter_path(old, new):
    'Compare the old dataset and the new one to find the compounds that are formed in an early generation'
    merged=pd.merge(old, new, on='ID', indicator='Common or not')
    merged=merged.loc[merged['Gen_x']>merged['Gen_y'], :]
    return merged



def compare_for_shorter_path_2(old, new):
    'Compare the old dataset and the new one to find the compounds that are formed in two early generation'
    merged=pd.merge(old, new, on='ID', indicator='Common or not')
    merged=merged.loc[merged['Gen_x']>merged['Gen_y']+1, :]
    return merged


def visualiser(dataframe):
    'visualises a molecule dataframe using rdkit'
    l_mol=[]
    for x in dataframe['ID']:
        l_mol.append(Chem.MolFromSmiles(x))
    return l_mol


def plotter(name, df_list):
    '''plots the generationwise compound count'''
    bars1=df_list[0].groupby('Gen').size().values.tolist()
    bars2=df_list[1].groupby('Gen').size().values.tolist()

    barWidth=0.4
    fig, ax = plt.subplots()

    r1=np.arange(len(bars1))

    x=ax.bar(r1-barWidth/2, bars1, color='blue', width=barWidth, edgecolor='white', label='Non-Iron Network')
    y=ax.bar(r1+barWidth/2, bars2, color='red', width=barWidth, edgecolor='white', label='Iron Network')

    def insert_data_labels(bars):
        for bar in bars:
            bar_height = bar.get_height()
            ax.annotate('{0:.0f}'.format(bar.get_height()),
                xy=(bar.get_x() + bar.get_width() / 2, bar_height),
                xytext=(5, 4),
                textcoords='offset points',
                ha='center',
                va='bottom'
            )




    insert_data_labels(x)
    insert_data_labels(y)
    plt.xlabel('Generations', fontweight='bold')
    plt.ylabel('Product Count', fontweight='bold')

    plt.title(f'{name} Reaction Network | Product Distribution by Generation')

    plt.savefig("proddist.png", format="png",transparent=True, dpi=300)

    plt.legend()
    plt.show()

def print_stats(name, df_list):
    print(f"{name} stats \n")
    print(f'No of compounds without Iron : '+str(len(df_list[0])))
    print(f'No of compounds with Iron : '+str(len(df_list[1])))
    print("   ")
    print(f'Compounds present in the Without Iron Network only: '+str(len(df_list[-2])))
    print(f'Compounds present in the With Iron Network only: '+str(len(df_list[-1])))
    print("   ")
    print(f'Total No of compounds being catalysed: '+str(len(compare_for_shorter_path(df_list[0], df_list[1]))))
    print(f'Percent of Original Dataset being catalysed : '+str(len(compare_for_shorter_path(df_list[0], df_list[1]))*100/len(df_list[0]))+" %")
    print("   ")
    print(f'No of compounds being catalysed by two generations: '+str(len(compare_for_shorter_path_2(df_list[0], df_list[1]))))

    
def calc_no_of_each_atoms(df):
    '''Calculates the descriptors for each compound of a dataframe, and returns a new dataframe.
        List of descriptor to add can be found here.
        https://mordred-descriptor.github.io/documentation/master/descriptors.html'''
    
    desc_needed = [ 'nN', 'nH', 'nC', 'nO','MW','nAcid','nBase','nHBAcc','nHBDon']
    calc = Calculator(descriptors, ignore_3D=False)
    calc.descriptors = [d for d in calc.descriptors if str(d) in desc_needed]
    mols = [Chem.MolFromSmiles(smi) for smi in df.ID.values.tolist()]
    calc_df=calc.pandas(mols)
    
    df['nC']=calc_df['nC']
    df['nH']=calc_df['nH']
    df['nO']=calc_df['nO']
    df['MW']=calc_df['MW']
    df['nAcid']=calc_df['nAcid']
    df['nBase']=calc_df['nBase']
    df['nHBAcc']=calc_df['nHBAcc']
    df['nHBDon']=calc_df['nHBDon']
    
    return df

def dist_atoms_gen(dflist,atom_type):
    '''calculates number of atoms in a molecule and returns condensed pivot dataframes.'''
    old_atoms=calc_no_of_each_atoms(dflist[0])
    new_atoms=calc_no_of_each_atoms(dflist[1])
    
    data_old=old_atoms.groupby(['Gen',atom_type]).size().unstack().fillna('-')
    data_new=new_atoms.groupby(['Gen',atom_type]).size().unstack().fillna('-')
    
    return [data_old,data_new]



def display_side_by_side(*args,titles=cycle([''])):
    '''displays molecules nicely in html'''
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

# Formose Reaction Network

In [ ]:
formose_data=load_data('formose')   # returns [withoutiron_df, withiron_df, Set_only_without_iron_comp, Set_only_iron_comp]
plotter('formose',formose_data)
print_stats('formose', formose_data)


In [ ]:
pyruvic_data=load_data('pyruvic')   # returns [withoutiron_df, withiron_df, Set_only_without_iron_comp, Set_only_iron_comp]
plotter('pyruvic',pyruvic_data)
print_stats('pyruvic', pyruvic_data)

In [ ]:
glucose_data=load_data('glucose')   # returns [withoutiron_df, withiron_df, Set_only_without_iron_comp, Set_only_iron_comp]
plotter('glucose',glucose_data)
print_stats('glucose', glucose_data)

In [ ]:
maillard=load_data('maillard')   # returns [withoutiron_df, withiron_df, Set_only_without_iron_comp, Set_only_iron_comp]
plotter('maillard',maillard)
print_stats('maillard', maillard)


# Calculating descriptors

In [ ]:
list_rxns=[formose_data, pyruvic_data, glucose_data, maillard]

list_compounds=[]

for x in list_rxns:
    for y in range(0,2):
        list_compounds.append(calc_no_of_each_atoms(x[y]))

In [ ]:
i=0
for x in list_rxns:
    i=i+1
    for y in range(0,2):
        x[y].to_csv(f'descriptor/compound{i}_{y}.csv')

# Help Section

In [ ]:
molecules=visualiser(compare_for_shorter_path_2(formose_data[0], formose_data[1]))
top_ten=[molecules[x] for x in range(0,4)]
Draw.MolsToGridImage(top_ten, subImgSize=(250,200))

# Distribution

In [ ]:
m=dist_atoms_gen(formose_data,'nC')

In [ ]:
display_side_by_side(m[0],m[1], titles=['Without Iron','With Iron'])